In [ ]:
!unzip '/content/FinalDataset.zip'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import csv
import matplotlib.pyplot as plt
from matplotlib.cbook import print_cycles
import os

import torchvision
from torchvision import transforms
import time
import matplotlib.pyplot as plt



#Artifical Neural Network Architecture
class ANNBaseline(nn.Module):
    def __init__(self):
        super(ANNBaseline, self).__init__()
        self.fc1 = nn.Linear(3*224*224,70)
        self.fc2 = nn.Linear(70, 30)
        self.fc3 = nn.Linear(30, 10)
        self.fc4 = nn.Linear(10, 6)


    def forward(self, x):
        x = x.view(-1, 3*224*224) #flatten feature data
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = (self.fc4(x)).squeeze()
        return x
        #print(x.shape)
        # y = torch.zeros(x.shape[0])
        # for i in range(x.shape[0]):
        #   y[i] = (x[i].max())
        # print(y.shape)
        # return y

def getLoaders(batch_size = 20):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomRotation([0, 360]),
        transforms.GaussianBlur(5),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.ToTensor()]
    )
    absPath = os.path.abspath("FinalDataset")
    training_path = absPath + "/train"
    validation_path = absPath + "/val"
    testing_path = absPath + "/test"
    train_set = torchvision.datasets.ImageFolder(root=training_path, transform=transform)
    validation_set = torchvision.datasets.ImageFolder(root=validation_path, transform=transform)
    testing_set = torchvision.datasets.ImageFolder(root=testing_path, transform=transform)
    

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader, test_loader

def get_accuracy(model, data_loader):

    correct = 0
    total = 0
    for imgs, labels in data_loader:
    
        if torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        
        output = model(imgs)
        
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
from matplotlib.path import lru_cache
def train(model, batch_size=20, num_epochs=1, lr=0.01, momentum=0.9):

    train_loader, val_loader, test_loader = getLoaders(batch_size);

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    train_loss, valid_loss, iters, train_acc, val_acc = [], [], [], [], []


    img_to_tensor = transforms.ToTensor()
    epochs = []
    
    for epoch in range(num_epochs):
        for img, labels in iter(train_loader):

          #To Enable GPU Usage
          if torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()

          optimizer.zero_grad()
          out = model((img))

          loss = criterion(out, labels)
          loss.backward()
          optimizer.step()
        train_loss.append(float(loss))

        for img, labels in iter(val_loader):
            out = model(img)
            loss = criterion(out, labels)
        valid_loss.append(float(loss))

        epochs.append(epoch)
        train_acc.append(get_accuracy(model, train_loader))
        val_acc.append(get_accuracy(model, val_loader))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
              epoch+1, loss, train_acc[-1], val_acc[-1]))
        
    # plotting
    plt.title("Training Curve")
    plt.plot(train_loss, label="Train")
    plt.plot(valid_loss, label="Valid")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, val_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [ ]:
model = ANNBaseline()
train(model, batch_size=57, num_epochs=20, lr=0.04)

In [ ]:
#Get testing accuracy
train_loader, val_loader, test_loader = getLoaders(40);
print("Accuracy:",get_accuracy(model, test_loader))

